In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoAlertPresentException

from time import sleep
from tqdm.notebook import tqdm
from tqdm import tqdm

import pandas as pd
import mysql.connector

In [5]:
def Sql_start(name, Password, DB):
    try:
        return mysql.connector.connect(
            host = "database-1.c3wcyagu0dve.ap-northeast-2.rds.amazonaws.com",
            port = "3306",
            user = name,
            password = Password,
            database = DB
        )
    except mysql.connector.Error as err:
        print("연결 실패: [err]")
        return None

In [7]:
DataBase = Sql_start("root", "Rrnrrn1!", "amrbase")
aws_cur = DataBase.cursor(buffered=True)

In [8]:
DataBase.close()

In [6]:
def Page_Open(Name):
    Page_link_num = driver.find_element(By.TAG_NAME,"nav")

    Page_run = Page_link_num.find_element(By.PARTIAL_LINK_TEXT, Name)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    sleep(0.5)
    Page_run.click()
    
# 기존 탭 저장
    original_window = driver.current_window_handle

# 현재 열려있는 모든 탭 핸들 가져오기
    windows = driver.window_handles

    return original_window, windows

In [7]:
def Cafe_Open(Name):
    Cafe = driver.find_element(By.PARTIAL_LINK_TEXT, Name)
    driver.execute_script("arguments[0].scrollIntoView(false);", Cafe)
    Cafe.click()
    
# 기존 탭 저장
    original_window = driver.current_window_handle

# 현재 열려있는 모든 탭 핸들 가져오기
    windows = driver.window_handles

# 새로 열린 탭으로 전환
    #driver.switch_to.window(windows[-1])
    #print("새 탭 타이틀:", driver.title)

    return original_window, windows

In [2]:
coffee_name = [
    '''//*[@id="contents"]/div/div/ul/li[2]''',
    '''//*[@id="contents"]/div/div/ul/li[8]''',
    '''//*[@id="contents"]/div/div/ul/li[9]'''    
]

In [3]:
def C_Bean(X_path):
    info_M = driver.find_element(By.XPATH, X_path)
    info_name = info_M.find_element(By.CLASS_NAME, "kor")
    info_list = info_M.find_element(By.CLASS_NAME, "info")
    info_list = info_list.find_elements(By.TAG_NAME, "dl")
    driver.execute_script("arguments[0].scrollIntoView(false);", info_M)
    
    return info_name, info_list

In [61]:
#웹 닫기
driver.quit()

In [ ]:
#탭 닫기
driver.close()
driver.switch_to.window(original_window)

In [9]:
#커피빈 커피 수집
Coffee_N = []
Kcal = []
Caffeine = []
CoffeeBean = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../../../driver/chromedriver"))
#driver = webdriver.Chrome(service=Service("../driver/chromedriver.exe"))
driver.get("https://www.coffeebeankorea.com/menu/list.asp?category=13")
sleep(0.5)

driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

#키워드 검색
Page_link = driver.find_element(By.CLASS_NAME, "paging")
Page_link = Page_link.find_element(By.LINK_TEXT, "1")
Page_link.click()
sleep(1)

for idx, Name in enumerate(coffee_name):
    if len(Coffee_N) == 1:
        Page_link = driver.find_element(By.CLASS_NAME, "paging")
        Page_link = Page_link.find_element(By.LINK_TEXT, "2")
        Page_link.click()
    
    info_name, info_list = C_Bean(Name)

    sleep(1)
    Coffee_N.append(info_name.text)
    for name in info_list:
        C_info = name.text.split('\n')[1]
        C_num = name.text.split('\n')[0] + " " + name.text.split('\n')[2]

        if C_info == "열량":
            Kcal.append(C_num)

        if C_info == "카페인":
            Caffeine.append(C_num)

    sleep(1)

CoffeeBean = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

driver.quit()
CoffeeBean

,Menu_name,Kcal,Caffeine
0,카페모카,305 Kcal,102 mg
1,아이스 아메리카노,5 Kcal,91 mg
2,아이스 카페라떼,119 Kcal,91 mg


In [ ]:
#MEGA 커피 수집
Coffee_N = []
Kcal = []
Caffeine = []
Mega = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../../../driver/chromedriver"))
#driver = webdriver.Chrome(service=Service("../driver/chromedriver.exe"))
driver.get("https://www.mega-mgccoffee.com/menu/?menu_category1=1&menu_category2=1")
sleep(0.5)

info_M = driver.find_element(By.XPATH, '''/html/body/div[4]/div[3]/div/div[3]/div[2]/div[3]''')
driver.execute_script("arguments[0].scrollIntoView(false);", info_M)
checkBox = info_M.find_elements(By.CLASS_NAME, "checkbox_wrap.list_checkbox")
sleep(0.5)
for check in checkBox:
    if check.text == "커피":
        print("커피 찾았어")
        check.click()
        break

sleep(0.5)
driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

move_page = driver.find_element(By.CLASS_NAME, "board_page")
move_page = move_page.find_elements(By.TAG_NAME, "li")

move_page[2].click()
sleep(0.5)

Menu_list = driver.find_element(By.CLASS_NAME, "cont_gallery_list.cont_gallery_list2.cont_list.cont_list4.cont_list_m.cont_list_m2.wfull")
Menu_list = Menu_list.find_elements(By.TAG_NAME, "li")
sleep(1)

#66 - 카페라떼 / 72-카페모카 / 114 - 아메리카노
C_Menu = [Menu_list[66], Menu_list[72], Menu_list[114]]

for C_menu_list in C_Menu:
    driver.execute_script("arguments[0].scrollIntoView(false);", C_menu_list)
    C_menu_list.click()

    C_menu_list2 = C_menu_list.find_element(By.CLASS_NAME, "inner_modal")

    Coffee_name = C_menu_list2.find_element(By.CLASS_NAME, "cont_text_inner.cont_text_title")
    Coffee_N.append(Coffee_name.text)

    Coffee_Kcal = C_menu_list2.find_elements(By.CLASS_NAME, "cont_text_inner")
    Kcal.append(Coffee_Kcal[3].text.split("제공량 ")[1])


    Coffee_Caffeine = C_Menu[0].find_element(By.CLASS_NAME, "cont_list.cont_list2.cont_list_small.cont_list_small2")
    Coffee_Caffeine = Coffee_Caffeine.find_elements(By.TAG_NAME, "li")
    Caffeine.append(Coffee_Caffeine[4].text.split(" ")[1])

Mega = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

Mega

커피 찾았어


,Menu_name,Kcal,Caffeine
0,카페라떼,145.1kcal,199.1mg
1,카페모카,294.4cal,199.1mg
2,아메리카노,12.2kcal,199.1mg


In [ ]:
#더벤티
Coffee_N = []
Kcal = []
Caffeine = []
Theventi = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../../../driver/chromedriver"))
#driver = webdriver.Chrome(service=Service("../driver/chromedriver.exe"))
driver.get("https://www.theventi.co.kr/new2022/menu/index.html")
sleep(0.5)

M_list = driver.find_element(By.CLASS_NAME, "sub-nav__wrap.sub-nav-clone--depth2")
driver.execute_script("arguments[0].scrollIntoView(false);", M_list)
M_list = M_list.find_element(By.LINK_TEXT, "전체메뉴")
sleep(0.5)

M_list.click()
sleep(0.5)

M_kind = driver.find_element(By.CLASS_NAME, "tab.box")
M_kind = M_kind.find_elements(By.TAG_NAME, "li")
M_kind[1].click()
sleep(0.5)

Menu_list = driver.find_element(By.CLASS_NAME, "menu_list")
Menu_list = Menu_list.find_elements(By.TAG_NAME, "a")
sleep(0.5)

# 1 = 아이스 아메리카노, 2 = 카페라떼, 10 = 카페모카
C_Menu = [Menu_list[1], Menu_list[2], Menu_list[10]]

for C_Menu_name in C_Menu:
    driver.execute_script("arguments[0].scrollIntoView(false);", C_Menu_name)
    C_Menu_name.click()
    sleep(0.5)

    C_Menu_list = driver.find_element(By.CLASS_NAME, "mfp-content")
    C_Menu_list2 = C_Menu_list.find_element(By.TAG_NAME, "p")
    C_Menu_list2 = C_Menu_list2.find_elements(By.TAG_NAME, "span")

    Coffee_N.append(C_Menu_list2[2].text.replace(" (벤티/더벤티)", ""))


    C_Menu_list1 = C_Menu_list.find_element(By.TAG_NAME, "tbody")
    C_Menu_list1 = C_Menu_list1.find_elements(By.TAG_NAME, "td")

#1 = 열량, 6 = 카페인
    Kcal.append(C_Menu_list1[1].text.replace(" (벤티/더벤티)", "") + "Kcal")
    Caffeine.append(C_Menu_list1[6].text.replace(" (고카페인)", "") + "mg")

    Close = C_Menu_list.find_element(By.CLASS_NAME, "xi-close")
    Close.click()
    sleep(0.5)

Theventi = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

driver.quit()
Theventi

,Menu_name,Kcal,Caffeine
0,아이스 아메리카노,11Kcal,192mg
1,카페라떼,160Kcal,296mg
2,카페모카,423Kcal,302mg


In [114]:
#컴포즈
Coffee_N = []
Kcal = []
Caffeine = []
compose = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../../../driver/chromedriver"))
#driver = webdriver.Chrome(service=Service("../driver/chromedriver.exe"))
driver.get("https://composecoffee.com/menu")
sleep(0.5)

M_list = driver.find_element(By.CLASS_NAME, "nav-item.dropdown.d-block.d-lg-none")
M_list.click()
sleep(0.5)

driver.execute_script("arguments[0].scrollIntoView(false);", M_list)
M_list = M_list.find_element(By.LINK_TEXT, "커피 · 더치")
sleep(0.5)

M_list.click()
sleep(0.5)

Menu_list = driver.find_elements(By.CLASS_NAME, "col-lg-3.col-md-4.col-sm-6.itemBox")
Menu_list_N = []

actions = ActionChains(driver)
for name in Menu_list:
    driver.execute_script("arguments[0].scrollIntoView(false);", name)
    sleep(0.5)

    if "ICE 아메리카노" in name.text:
        actions.move_to_element(name).perform()
        Menu_list_N.append(name.text.split("\n"))
        sleep(0.5)
    elif "ICE 카페라떼" in name.text:
        actions.move_to_element(name).perform()
        Menu_list_N.append(name.text.split("\n"))
        sleep(0.5)
    elif "ICE 카페모카" in name.text:
        actions.move_to_element(name).perform()
        Menu_list_N.append(name.text.split("\n"))
        sleep(0.5)

#0 = name / 4 = Kcal / 2 = 카페인
for name in Menu_list_N:
    Coffee_N.append(name[0])
    Kcal.append((name[4].split(" "))[3] + "Kcal")
    Caffeine.append((name[2].split(" ")[5]).split("/")[0])

compose = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

driver.quit()
compose

,Menu_name,Kcal,Caffeine
0,ICE 아메리카노,15Kcal,156mg
1,ICE 카페라떼,155Kcal,208mg
2,ICE 카페모카,321Kcal,208mg


In [ ]:
driver = webdriver.Chrome(service=Service("../../../driver/chromedriver"))
#driver = webdriver.Chrome(service=Service("../driver/chromedriver.exe"))
driver.get("https://paikdabang.com/menu/menu_new/")
sleep(0.5)

M_list = driver.find_element(By.CLASS_NAME, "page_tab.st-2")
M_list = M_list.find_element(By.LINK_TEXT, "커피")
M_list.click()
sleep(0.5)

Menu_list = driver.find_element(By.CLASS_NAME, "menu_list.clear")
Menu_list = Menu_list.find_elements(By.CLASS_NAME, "li")

for M_name in Menu_list:
    print(M_list.text)


In [143]:
Coffee_N = []
Kcal = []
Caffeine = []
paikdabang = pd.DataFrame()

driver = webdriver.Chrome(service=Service("../../../driver/chromedriver"))
#driver = webdriver.Chrome(service=Service("../driver/chromedriver.exe"))
driver.get("https://paikdabang.com/menu/menu_new/")
sleep(0.5)

M_list = driver.find_element(By.CLASS_NAME, "page_tab.st-2")
M_list = M_list.find_element(By.LINK_TEXT, "커피")
M_list.click()
sleep(0.5)

Menu_list = driver.find_element(By.CLASS_NAME, "menu_list.clear")
Menu_list = Menu_list.find_elements(By.CLASS_NAME, "li")

Menu_list = driver.find_element(By.CLASS_NAME, "menu_list.clear")
driver.execute_script("arguments[0].scrollIntoView(false);", Menu_list)
sleep(0.5)

Menu_list = Menu_list.find_elements(By.TAG_NAME, "li")

Menu_list_N = []

for idx, M_name in enumerate(Menu_list):

    if "아메리카노(ICED)" == M_name.text:
        driver.execute_script("arguments[0].scrollIntoView(false);", M_name)
        sleep(1)
        M_name.click()
        sleep(0.5)

        Coffee_N.append(M_name.text.split("\n")[0].replace("(ICED)", ""))
        Kcal.append(M_name.text.split("\n")[8]+ "Kcal")
        Caffeine.append(M_name.text.split("\n")[6]+"mg")
        sleep(0.5)

        Menu_list_N.append(M_name.text)

    elif "카페라떼(ICED)" == M_name.text:
        driver.execute_script("arguments[0].scrollIntoView(false);", M_name)
        sleep(1)
        M_name.click()
        sleep(0.5)

        Coffee_N.append(M_name.text.split("\n")[0].replace("(ICED)", ""))
        Kcal.append(M_name.text.split("\n")[9]+ "Kcal")
        Caffeine.append(M_name.text.split("\n")[7]+"mg")
        sleep(0.5)

        Menu_list_N.append(M_name.text)

    elif "카페모카(ICED)" == M_name.text:
        driver.execute_script("arguments[0].scrollIntoView(false);", M_name)
        sleep(1)
        M_name.click()
        sleep(0.5)

        Coffee_N.append(M_name.text.split("\n")[0].replace("(ICED)", ""))
        Kcal.append(M_name.text.split("\n")[10]+ "Kcal")
        Caffeine.append(M_name.text.split("\n")[8]+"mg")
        sleep(0.5)

        Menu_list_N.append(M_name.text)

paikdabang = pd.DataFrame({
    "Menu_name": Coffee_N,
    "Kcal": Kcal,
    "Caffeine": Caffeine
})

driver.quit()
paikdabang

,Menu_name,Kcal,Caffeine
0,아메리카노,13Kcal,237mg
1,카페라떼,198Kcal,237mg
2,카페모카,365Kcal,237mg
